In [3]:
"""2021年1月30日22:46:19 更新:
加了个 baidu 搜索。代码太烂了，估计不会再改什么了。
baidu 搜索还要 COOKIE，简直了 2021年1月30日23:26:09"""
import re
import time    
    
import requests

# baidu 的两个 COOKIE 项，如果要用百度搜索的话是必须的
BAIDUID = ''
BIDUPSID = ''

class Net(object):
        
    def request(self, method, url, headers=None, params=None, data=None, **kwargs):
        _headers = {'user-agent': 'wasp', 'referer': url}
        
        if headers is None:
            headers = _headers
        
        if 'user-agent' not in headers:
            headers.update({'user-agent': 'wasp'})
            
        return requests.request(method, url, headers=headers, params=params, data=data, **kwargs)
    
    def get(self, url, headers=None, params=None, **kwargs):
        return self.request('GET', url, headers=headers, params=params, data=None, **kwargs)
    
    def post(self, url, headers=None, data=None, **kwargs):
        return self.request('POST', url, headers=headers, params=None, data=data, **kwargs)

    
class SearchHelper(object):
    """
    搜索方法返回一个列表，内含如下格式的字典:
    
    {
        'author': '幻想乡的新月',
        'mid': 149592,
        'aid': 'av2600096',
        'title': '【东方MMD】暗黑炼狱火☆',
        'description': 'sm26747660 这个简直wwww~作者FSM'
    }
  """
    def __init__(self, SESSDATA='', BILI_JCT=''):
        self.found_aid = set() # 找到的视频 av/bv 号集合
        self.not_found_kw = set() # 没有找的的关键字集合
        self.result_list = []
        
        self.net = Net()
        
        self.cookie = {}
        
        self.bili_headers= {
            'origin': 'https://search.bilibili.com',
            'referer': 'https://search.bilibili.com/',
            'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                           ' Chrome/87.0.4280.88 Safari/537.36 Edg/87.0.664.66'),
            'cookie': 'SESSDATA={}; BILI_JCT={};'.format(SESSDATA, BILI_JCT)
        }
        
    def _find_key(self, dict_obj, kw_set, kw_map=None, default_value=None):
        """提取出字典中指定的 keyword 组成一个一维的字典，并根据可选的 keyword map 将原字典的指定键映射到给出的键"""
        def dfs(dict_obj, result_dict):
            """简单的 DFS，把 dict 中 key in kw_set 的键值对 update 到 result_dict 里面"""
            for k, v in dict_obj.items():
                if k in kw_set: 
                    result_dict.update({k: v})
                else: 
                    if isinstance(v, list):
                        for i in v:
                            if isinstance(i, dict):
                                dfs(i, result_dict)

                    if isinstance(v, dict):
                        dfs(v, result_dict)
    
        result_dict = {}
        dfs(dict_obj, result_dict)
        
        for key in list(kw_set):
            if key not in result_dict:
                result_dict.update({key: default_value})
        
        if kw_map is None:
            return result_dict
    
        mapped_dict = {}
        for key in kw_set:
            if key in kw_map: # 如果关键字在映射表中，改键名
                mapped_dict.update({kw_map[key]: result_dict[key]})
            else: # 不在就不修改
                mapped_dict.update({key: result_dict[key]})
        
        return mapped_dict
        
    def search(self, kw_list, desc_regex='', doge=True, baidu=False):
        """聚合站内检索和站外检索方法，先站内检索，后站外检索
        返回一个元组，第一个元素值为视频信息 dict（格式见类的注释），第二个元素值为未找到的关键字列表
        ([{...}], [...])"""
        import traceback
        
        try:
            self.not_found_kw = set(kw_list) # 初始假设所有关键字都没有找到
            
            kw_list = list(self.not_found_kw) # 去重然后还原
            
            print('*** 开始站内检索，预计耗时 {}s'.format(2 * len(kw_list)))
            
            found, not_found = self.search_from_bili(kw_list, desc_regex=desc_regex)
            print('找到 {} 个数据'.format(len(found)))
            
            if doge:
                print()
                print('*** 开始站外检索(Dogedoge)，预计耗时 {}s'.format(2 * len(not_found)))

                found, not_found = self.search_from_doge(list(self.not_found_kw), desc_regex=desc_regex)
                print('找到 {} 个数据'.format(len(found)))
            if baidu:
                print()
                print('*** 开始站外检索(Baidu)，预计耗时 {}s'.format(2 * len(not_found)))
                
                found, not_found = self.search_from_baidu(list(self.not_found_kw), desc_regex=desc_regex)
                print('找到 {} 个数据'.format(len(found)))
        except Exception:
            print('#' * 30)
            print('ERROR:')
            traceback.print_exc()
        
        print()
        
        return self.result_list, list(self.not_found_kw)
    
    # 站内检索
    def search_from_bili(self, kw_list, desc_regex=''):
        """站内检索
        提供一个 desc_regex 正则表达式提取简介中符合条件的 keyword list => desc_words，如果 desc_words[0] == keyword，确定该视频
        是要找的视频，否则该视频可能只是和 keyword 关键字相关而被检索到，不移除 not_found_kw 中对应的值"""
        
        headers =self.bili_headers
        
        kw_list = list(set(kw_list))
        kw_count = len(kw_list)
        
        # 根据数据量动态设置休眠时间。不知道有没有用
        if kw_count < 40:
            delay = 0.5
        elif kw_count >= 40 and kw_count <= 80:
            delay = 1.5
        else:
            delay = 2.5
                    
        for kw in kw_list:
            api = 'https://api.bilibili.com/x/web-interface/search/type'
            print('Searching {} ...'.format(kw))
            
            for page in range(1, 6): # 最多取 5 页数据
                params = {
                    'keyword': kw,
                    'search_type': 'video',
                    'page': page
                }

                dict_kw_set = {'aid', 'author', 'mid', 'title', 'description'}
                jsoup = self.net.get(api, headers=headers, params=params).json()

                if jsoup['code'] == 0: 
                    data = jsoup['data']
                    
                    if 'result' in data:
                        for item in data['result']:
                            dict_item = {'keyword': kw}
                            aid = 'av' + str(item['aid'])

                            if aid not in self.found_aid:
                                self.found_aid.add(aid)

                                dict_item.update(self._find_key(item, dict_kw_set))
                                dict_item.update({'aid': aid})

                                for k, v in dict_item.items(): # 转 str
                                    dict_item[k] = str(v)
                                    
                                self.result_list.append(dict_item)

                                if kw in self.not_found_kw:
                                    if desc_regex:
                                        desc_words = re.findall(desc_regex, dict_item['description'])
                                        if desc_words and (desc_words[0] == dict_item['keyword']): # 如果关键字存在且为第一个项，认为找到了
                                            self.not_found_kw.remove(kw)
                                    else:
                                        self.not_found_kw.remove(kw)
                                    
                        print('休眠 {}s'.format(delay))
                        time.sleep(delay) 
                        
                        # 一页数据全部获取到后就退出             
                        break
                    else:
                        self.not_found_kw.add(kw)
                        print(kw, '无数据，可能是因为被屏蔽或被删除')
                        break
                # 被 ban，似乎其他接口没受到影响
                # 恢复时间大概 30min ?
                # {'code': -412, 'message': '请求被拦截', 'ttl': 1, 'data': None} 
                # elif jsoup['code'] == -412: 
                else:
                    print('接口调用过于频繁，请稍后再试')
                    print(jsoup)
                    break
                
        # 考虑到单独使用方法可能
        return self.result_list, list(self.not_found_kw)
    
    # 剩下的全是站外检索相关方法
    def get_video_info(self, vid):
        """获取视频信息 json"""
        if vid.startswith('BV'):
            bapi = 'https://api.bilibili.com/x/web-interface/view?bvid={}'
            video_info = self.net.get(bapi.format(vid), headers=self.bili_headers).json()
        else: # 以 AV 号为参数的视频信息接口， dogedoge 返回的基本都是 BV 号，不过还加上以防万一
            aapi = 'https://api.bilibili.com/x/web-interface/view?aid={}'
            video_info = self.net.get(aapi.format(vid[2:]), headers=self.bili_headers).json()
            
        return video_info
    
    # 站外检索专用
    def add_to_resultlist(self, video_info, vid, kw):
        """把结果添加到列表"""
        dict_item = {}
        if video_info['code'] == -403: # 这种稿件是登录可见的 {'code': -403, 'message': '访问权限不足', 'ttl': 1}
            if vid.startswith('BV'): # 如果是 BV 号，转成 AV 号
                stat = 'https://api.bilibili.com/x/web-interface/archive/stat?bvid={}'
                vid = 'av' + str(self.net.get(stat.format(vid)).json()['data']['aid'])
                
            print(kw, vid, '需要登录查看')

            dict_item = {
                    'keyword': kw, 'aid': vid, 
                    'author': 'unknown', 'mid': -1, 
                    'description': '视频需要登录查看', 'title': '视频需要登录查看'
            }
            
            self.result_list.append(dict_item) 
        elif video_info['code'] == 0: # 正常
            data = video_info['data']
            dict_item = {'keyword': kw}
            dict_kw_set = {'aid', 'name', 'mid', 'desc', 'title'}
            dict_item.update(self._find_key(data, dict_kw_set, {'name': 'author', 'desc': 'description'}))
            dict_item['aid'] = 'av' + str(dict_item['aid'])

            for k, v in dict_item.items(): # 全部转为 str
                dict_item[k] = str(v)

            if dict_item['aid'] not in self.found_aid:
                self.found_aid.add(dict_item['aid'])                        

                self.result_list.append(dict_item)
        else: # 到这里说明稿件已经被删除
            if video_info['code'] == -404:
                print(kw, '已被削除')
            else:
                print('检索 {} 失败，MSG: {} CODE:{}'.format(kw, video_info['message'], video_info['code']))

            self.not_found_kw.add(kw)
        
        return dict_item
    
    # dogedoge
    def search_from_doge(self, kw_list, desc_regex=''):       
        from lxml.html import fromstring
        
        url = 'https://www.dogedoge.com/results'
        
        headers = {
            'referer': 'https://www.dogedoge.com/',
            'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                           ' Chrome/87.0.4280.88 Safari/537.36 Edg/87.0.664.66')
        }
        
        kw_list = list(set(kw_list)) # 去重
        kw_count = len(kw_list)
        
        if kw_count <= 20: # 数据量在 20 个以内时最多休眠 2s
            max_delay = 2
        else:
            max_delay = 3 # 数据量超过 20 最多休眠 3s
        
        for kw in kw_list:
            params = {
                'q': kw,
                'lang': 'cn'
            }
            
            html = self.net.get(url, headers=headers, params=params).text
            elements = fromstring(html)
            element_a = elements.xpath('//a[@class="result__url js-result-extras-url"]')
            
            print('Searching {} ...'.format(kw))
            
            if not element_a:
                self.not_found_kw.add(kw)
            
            for a in element_a[:5]: # 取前五个数据
                domain = a.xpath('string(./span[@class="result__url__domain"])')
                
                if domain and domain.find('www.bilibili.com') != -1: # 是 B 站的内容
                    redirect_url = 'https://www.dogedoge.com/' + a.get('href')
                    dist = self.net.get(redirect_url, headers=headers, allow_redirects=False).headers['location']
                                        
                    vid = re.findall('BV\w+|av\d+', dist)
                    
                    if vid:
                        vid = vid[0]
                        
                        video_info = self.get_video_info(vid)
                        dict_item = self.add_to_resultlist(video_info, vid, kw)
                        if not dict_item: # 检索失败，break
                            break
                            
                        if kw in self.not_found_kw:
                            if desc_regex:
                                desc_words = re.findall(desc_regex, dict_item['description'])
                                if desc_words and desc_words[0] == dict_item['keyword']: # 如果关键字存在且为第一个项，认为找到了
                                    self.not_found_kw.remove(kw)
                            else:
                                self.not_found_kw.remove(kw)
                       
                        # 取所有相关的数据，每次休眠 0.3s
                        time.sleep(0.3)
                    
            from random import randint
            
            delay = randint(1, max_delay) # 随机暂停 1-max_delay s
            print('休眠 {}s.'.format(delay))            
            time.sleep(delay)
        
        # 考虑到单独使用方法可能
        return self.result_list, list(self.not_found_kw)
    
    # baidu
    def search_from_baidu(self, kw_list, desc_regex=''):
        from lxml.html import fromstring
        
        url = 'https://www.baidu.com/s?si=bilibili.com&ie=utf-8' # 指定只搜索B站内容
        
        headers = {
            'host': 'www.baidu.com',
            'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                           ' Chrome/87.0.4280.88 Safari/537.36 Edg/87.0.664.66'),
            'Cookie': '''BAIDUID={BAIDUID}; BIDUPSID={BIDUPSID};'''.format(BAIDUID, BIDUPSID)
       }
        
        kw_list = list(set(kw_list)) # 去重
        kw_count = len(kw_list)
        
        if kw_count <= 20: # 数据量在 20 个以内时最多休眠 2s
            max_delay = 1
        else:
            max_delay = 3 # 数据量超过 20 最多休眠 3s
        
        for kw in kw_list:
            params = {
                'wd': kw
            }
            
            print('Searching {} ...'.format(kw))
            
            html = self.net.get(url, headers=headers, params=params).text
            
            try:
                elements = fromstring(html)
                result_div = elements.xpath('//div[@class="result c-container new-pmd"]')
            except Exception as e:
                self.not_found_kw.add(kw)
                continue
            
            if not result_div:
                self.not_found_kw.add(kw)
            
            for elem in result_div[:5]: # 取前 5 个数据
                # show_url = elem.xpath('.//span[@class="c-showurl"]/text()')
                location = elem.xpath('.//h3[@class="t"]/a/@href')

                if location: # and show_url and show_url[0].find('www.bilibili.com') != -1: 这里已经限制了是B站检索，所以不用判断是否是B站内容
                    ua_header = {'user-agent': headers['user-agent']}
                    vid = self.net.get(location[0], ua_header, allow_redirects=False).headers['location'] # 没有 headers（用默认headers）
                    vid = re.findall('BV\w+|av\d+', vid)
                    
                    if vid:
                        vid = vid[0]
                        
                        video_info = self.get_video_info(vid)
            
                        dict_item = self.add_to_resultlist(video_info, vid, kw)
                        if not dict_item: # 检索失败，break
                            break
                            
                        if kw in self.not_found_kw:
                            if desc_regex:
                                desc_words = re.findall(desc_regex, dict_item['description'])
                                if desc_words and desc_words[0] == dict_item['keyword']: # 如果关键字存在且为第一个项，认为找到了
                                    self.not_found_kw.remove(kw)
                            else:
                                self.not_found_kw.remove(kw)
    
                        time.sleep(0.3)
                    
            from random import randint

            delay = randint(1, max_delay) # 随机暂停 1-max_delay s
            print('休眠 {}s.'.format(delay))            
            time.sleep(delay)
                
        return self.result_list, list(self.not_found_kw)
                
                
def format_result(result, only_one=False):
    """完全为了检索 sm 号而准备"""
    sus = [] # 疑似相关视频列表
    repeated = [] # 重复视频列表
    
    # 记录所有关键字和完全匹配的关键字
    # 有些关键字可能只是因为有视频提到了而被误认为被找到了，然而实际上可能因为各种原因并没有被搬运
    all_keyword = set()
    certain_keyword = set()
    
    found, not_found = result
    
    found.sort(key=lambda x:x['keyword'])

    print()
    print(' 检索结果 '.center(40, '='))
    for item in found:
        content = ' '.join([item['keyword'], str(item['aid']), '(' + item['title'] + ')', item['author'] + '(uid:' + str(item['mid']) + ')'])
        
        all_keyword.add(item['keyword'])
        
        # 提取简介中的 sm 号
        parsed_kw = re.findall('sm\d+', item['description'])
        
        # 如果没有 sm 号 或 首个 sm 号和 keyword 不同，加入疑似列表
        # 第二个条件判断第一个 sm 号是否和关键字对应是基于搬运的格式，如果搬运给出的第一个 sm 号不是视频原地址就可能出错，
        # 不过 B 站搬运格式貌似是固定的，间接第一行就是 original link
        if (not parsed_kw) or (parsed_kw[0] != item['keyword']):
            sus.append((content + '\n简介:\n' + item['description']))            
        else: # 否则这里输出的内容都是确定的 sm -> av
            if item['keyword'] not in certain_keyword: # sm 号已经出现过，表明检索结果有重复
                certain_keyword.add(item['keyword'])
                print(content)
            else:
                if only_one:
                    repeated.append(content)
                else:
                    print(content)            

                    
    print()
    print('未找到的数据:')
    print(not_found)
    
    print()
    print('可能没找到的数据:')
    print(list(all_keyword - certain_keyword))
    
    if only_one and repeated:
        print()
        print('重复出现过的数据:')
        for c in repeated:
            print(c)
    
    if sus:
        print()
        print('相关数据:')        
        for s in sus:
            print('*' * 20)
            print(s)
            print('*' * 20)
            print()

In [4]:
# 填入你的B站的 SESSDATA 和 BILI_JCT。非必须，填了的话能够获取需要登录的视频的信息。
SESSDATA = ''
BILI_JCT = ''

In [ ]:
text = """要检索的 sm 号放在这里"""
li = re.findall('sm\d+', text)

st = time.time()

sh = SearchHelper(SESSDATA=SESSDATA, BILI_JCT=BILI_JCT)

result = sh.search(li, desc_regex='sm\d+', doge=True, baidu=True)

format_result(result, only_one=True)

print()
print('耗时 {}s'.format(round(time.time() - st, 2)))

In [1]:
format_result(result, only_one=True)

In [ ]:
# 下面的代码可以不用看

In [41]:
def format_result(ori_text, result, only_one=False):
    """完全为了检索 sm 号而准备"""
    sus = [] # 疑似相关视频列表
    repeated = [] # 重复视频列表
    res_list = []
    
    letter_list = ori_text.split()
    
    # 记录所有关键字和完全匹配的关键字
    # 有些关键字可能只是因为有视频提到了而被误认为被找到了，然而实际上可能因为各种原因并没有被搬运
    all_keyword = set()
    certain_keyword = set()
    
    found, not_found = result
    
    found.sort(key=lambda x:x['keyword'])

    print()
    print(' 检索结果 '.center(40, '='))
    for item in found:
        content = ' '.join([item['keyword'], str(item['aid']), '(' + item['title'] + ')', item['author'] + '(uid:' + str(item['mid']) + ')'])
        
        all_keyword.add(item['keyword'])
        
        # 提取简介中的 sm 号
        parsed_kw = re.findall('sm\d+', item['description'])
        
        # 如果没有 sm 号 或 首个 sm 号和 keyword 不同，加入疑似列表
        # 第二个条件判断第一个 sm 号是否和关键字对应是基于搬运的格式，如果搬运给出的第一个 sm 号不是视频原地址就可能出错，
        # 不过 B 站搬运格式貌似是固定的，间接第一行就是 original link
        if (not parsed_kw) or (parsed_kw[0] != item['keyword']):
            sus.append((content + '\n简介:\n' + item['description']))            
        else: # 否则这里输出的内容都是确定的 sm -> av
            if item['keyword'] not in certain_keyword: # sm 号已经出现过，表明检索结果有重复
                certain_keyword.add(item['keyword'])
                for v in letter_list:
                    if v.find(item['keyword']) != -1:
                        res_list.append(' '.join([v, item['aid']]))
                        break
            else:
                if only_one:
                    repeated.append(content)
                else:
                    print(content)            

    
    def sort_key(x):
        rank = x[:x.find(':')]
        
        try:
            return int(rank)
        except:
            return -1
    
    res_list.sort(key=sort_key, reverse=True)
    
    for content in res_list:
        print(content)
    
    print()
    print('未找到的数据:')
    print(not_found)
    
    print()
    print('可能没找到的数据:')
    print(list(all_keyword - certain_keyword))
    
    if only_one and repeated:
        print()
        print('重复出现过的数据:')
        for c in repeated:
            print(c)
    
    if sus:
        print()
        print('相关数据:')        
        for s in sus:
            print('*' * 20)
            print(s)
            print('*' * 20)
            print()

In [42]:
ori = """N的月刊文本"""
format_result(ori, result, only_one=True)


================= 检索结果 =================
去年1位:sm34646576 av43844117
去年2位:sm34911065 av96935823
去年3位:sm35030158 av50562926
P3(タグ無し):sm36264452 av413833644
P7(タグ無し):sm36366125 av89045825
P5(91位):sm36447572 av93619689
P9(102位):sm36544132 av97998450
P10(55位):sm36559118 av98722457
P11(タグ無し):sm36694382 av837800706
P12(84位):sm37278793 av884183155
P15(56位):sm37309546 av926652399
P14(タグ無し):sm37371089 av926777818
OP(105位):sm37636346 av244869949
P4(127位):sm37657468 av585199594
P6(70位):sm37658465 av542382676
ED(284位):sm37766879 av970171849
1:sm36199151 av82697970
2:sm36310499 av86499162
3:sm37658498 av969920256
4:sm37096692 av286219009
5:sm36520023 av96632158
7:sm36779158 av925386980
8:sm36212103 av87744945
9:sm36983297 av243440502
11:sm36923321 av498324283
12:sm37914689 av713099844
14:sm36183565 av82125740
15:sm37536806 av499577466
16:sm36321377 av86836860
17:sm37474552 av669482668
20:sm37546901 av839700115
21:sm37458477 av884427566
22:sm37914133 av800606431
23:sm37218754 av456497923
24:sm365922

In [17]:
# 月刊文本解析
text = """1:sm36199151　11:sm36923321　21:sm37458477　31:sm36227304　41:sm36365537
2:sm36310499　12:sm37914689　22:sm37914133　32:sm37677676　42:sm36435393
3:sm37658498　13:sm37219270　23:sm37218754　33:sm37543440　43:sm36216301
4:sm37096692　14:sm36183565　24:sm36592289　34:sm36230202　44:sm36456829
5:sm36520023　15:sm37536806　25:sm36523715　35:sm37168497　45:sm37731430
6:sm37719055　16:sm36321377　26:sm36183546　36:sm37278315　46:sm36621160
7:sm36779158　17:sm37474552　27:sm36775111　37:sm36246271　47:sm36269325
8:sm36212103　18:sm36389325　28:sm37463898　38:sm36280754　48:sm36833345
9:sm36983297　19:sm36654811　29:sm37231202　39:sm36365740　49:sm37632038
10:sm36180987　20:sm37546901　30:sm37425718　40:sm36747136　50:sm37711052

OP(105位):sm37636346 ED(284位):sm37766879
P1(51位):sm36454632 P2(タグ無し):sm36509033 P3(タグ無し):sm36264452
P4(127位):sm37657468 P5(91位):sm36447572 P6(70位):sm37658465
P7(タグ無し):sm36366125 P8(タグ無し):sm36519124 P9(102位):sm36544132
P10(55位):sm36559118 P11(タグ無し):sm36694382 P12(84位):sm37278793
P13(247位):sm37942259 P14(タグ無し):sm37371089 P15(56位):sm37309546
去年1位:sm34646576 去年2位:sm34911065 去年3位:sm35030158

バックナンバー:mylist/66036945
2019年→sm36230202 12月号→sm38099680
"""

import re

li = re.findall('(.+?):(sm\d+).?', text)
def sort_key(x):
    try:
        return int(x[0])
    except:
        return -1
    
li.sort(key=sort_key, reverse=True)